In [1]:
import gradio as gr
import numpy as np
import os
import tempfile
import json
import requests
from datetime import datetime
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader, DirectoryLoader, PyPDFLoader
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI, HuggingFaceHub
import speech_recognition as sr
from gtts import gTTS
from transformers import pipeline
from translate import Translator
import logging
import pandas as pd
import matplotlib.pyplot as plt
from io import BytesIO
import base64
from fuzzywuzzy import fuzz
import calendar

: 

In [ ]:
# Données JSON (50 questions/réponses)
qa_data = [
    {"question": "Quels sont les horaires d'ouverture de la bibliothèque de l'ENIAD Berkane ?", "réponse": "La bibliothèque est ouverte de 8h à 20h du lundi au vendredi, et de 9h à 17h le samedi."},
    {"question": "Comment s'inscrire à un cours d'intelligence artificielle ?", "réponse": "L'inscription se fait en ligne via le portail étudiant ou au bureau des admissions de l'ENIAD."},
    {"question": "Où se trouve le campus de l'ENIAD Berkane ?", "réponse": "Le campus est situé à Berkane, près de l'Université Mohammed 1er, dans la région de l'Oriental."},
    {"question": "Quelles sont les filières proposées à l'ENIAD ?", "réponse": "L'ENIAD propose des filières en IA, Robotique, Sécurité Informatique et Génie Informatique."},
    {"question": "Quel est le coût des frais de scolarité ?", "réponse": "Les frais varient selon le programme ; contactez l'administration pour plus de détails."},
    {"question": "Y a-t-il un internat à l'ENIAD Berkane ?", "réponse": "Non, l'ENIAD ne dispose pas d'internat, mais des logements privés sont disponibles à proximité."},
    {"question": "Comment contacter un professeur ?", "réponse": "Les coordonnées des professeurs sont disponibles sur le portail étudiant ou au secrétariat."},
    {"question": "Quand commence l'année scolaire 2025-2026 ?", "réponse": "L'année scolaire débute généralement en septembre, vérifiez les dates exactes sur le site officiel."},
    {"question": "Quels sont les prérequis pour le cycle ingénieur ?", "réponse": "Un diplôme de niveau bac+2 ou équivalent dans une filière technique est requis."},
    {"question": "L'ENIAD offre-t-elle des formations en ligne ?", "réponse": "Oui, certaines formations courtes sont disponibles en ligne via la plateforme e-learning."},
    {"question": "Comment accéder au Wi-Fi du campus ?", "réponse": "Utilisez votre identifiant étudiant et mot de passe fournis à l'inscription."},
    {"question": "Quels logiciels sont utilisés en cours d'IA ?", "réponse": "Python, TensorFlow, PyTorch et d'autres outils sont couramment utilisés."},
    {"question": "Y a-t-il des stages obligatoires ?", "réponse": "Oui, un stage est requis en fin de cycle pour valider le diplôme d'ingénieur."},
    {"question": "Où puis-je trouver le calendrier des examens ?", "réponse": "Le calendrier est publié sur le portail étudiant au début de chaque semestre."},
    {"question": "L'ENIAD organise-t-elle des événements ?", "réponse": "Oui, des hackathons, conférences et ateliers sont régulièrement organisés."},
    {"question": "Comment demander une bourse ?", "réponse": "Soumettez une demande via le portail étudiant avant la date limite annuelle."},
    {"question": "Quels sont les débouchés après un diplôme de l'ENIAD ?", "réponse": "Les diplômés travaillent dans l'IA, la cybersécurité, le développement logiciel, etc."},
    {"question": "Y a-t-il un club de robotique ?", "réponse": "Oui, le club de robotique est ouvert à tous les étudiants intéressés."},
    {"question": "Comment réserver une salle de travail ?", "réponse": "Les réservations se font via l'application interne de l'ENIAD."},
    {"question": "Quels sont les horaires des cours ?", "réponse": "Les cours ont lieu généralement de 8h30 à 17h, selon l'emploi du temps."},
    {"question": "L'ENIAD a-t-elle un partenariat avec des entreprises ?", "réponse": "Oui, elle collabore avec des entreprises technologiques pour des projets et stages."},
    {"question": "Comment assister à une journée portes ouvertes ?", "réponse": "Les dates sont annoncées sur le site officiel ; l'inscription est souvent requise."},
    {"question": "Quels sont les documents pour l'inscription ?", "réponse": "Carte d'identité, diplômes, relevés de notes et photos d'identité sont nécessaires."},
    {"question": "Y a-t-il des cours en anglais ?", "réponse": "Oui, certains modules avancés sont dispensés en anglais."},
    {"question": "Comment signaler un problème technique ?", "réponse": "Contactez le support informatique via l'adresse email support@eniad.ump.ma."},
    {"question": "L'ENIAD propose-t-elle des certifications ?", "réponse": "Oui, des certifications en IA et cybersécurité sont disponibles."},
    {"question": "Où manger sur le campus ?", "réponse": "Une cafétéria est disponible, mais il n'y a pas de restaurant universitaire."},
    {"question": "Comment participer à un projet de recherche ?", "réponse": "Contactez les professeurs ou le département de recherche de l'ENIAD."},
    {"question": "Quels sont les délais pour payer les frais ?", "réponse": "Les frais doivent être payés avant le début de chaque semestre."},
    {"question": "L'ENIAD a-t-elle une bibliothèque numérique ?", "réponse": "Oui, une bibliothèque numérique est accessible avec vos identifiants étudiants."},
    {"question": "Qu'est-ce que le Big Data à l'ENIAD Berkane ?", "réponse": "Le Big Data à l'ENIAD couvre l'analyse de grandes quantités de données avec des outils comme Hadoop et Spark."},
    {"question": "Quels outils de Big Data sont enseignés ?", "réponse": "Hadoop, Apache Spark, Kafka et des bases de données NoSQL comme MongoDB sont au programme."},
    {"question": "Comment s'inscrire au module Big Data ?", "réponse": "Inscrivez-vous via le portail étudiant ou consultez le responsable du département IA."},
    {"question": "Y a-t-il des projets Big Data à l'ENIAD ?", "réponse": "Oui, les étudiants travaillent sur des projets réels avec des entreprises partenaires."},
    {"question": "Quelles sont les bases de données utilisées en Big Data ?", "réponse": "Les bases vectorielles, MongoDB, et Cassandra sont souvent utilisées."},
    {"question": "Comment accéder aux serveurs Big Data ?", "réponse": "Les serveurs sont accessibles via une connexion sécurisée avec vos identifiants étudiants."},
    {"question": "Le Big Data est-il obligatoire à l'ENIAD ?", "réponse": "Non, mais il est fortement recommandé pour les spécialisations en IA et informatique."},
    {"question": "Quels sont les prérequis pour le cours de Big Data ?", "réponse": "Des bases en programmation (Python) et en bases de données sont nécessaires."},
    {"question": "Y a-t-il des certifications en Big Data ?", "réponse": "Oui, l'ENIAD propose des certifications reconnues en partenariat avec des entreprises."},
    {"question": "Comment analyser des données massives à l'ENIAD ?", "réponse": "Utilisez les outils enseignés comme Spark ou participez aux ateliers pratiques."},
    {"question": "Quels sont les débouchés du Big Data ?", "réponse": "Data Scientist, Ingénieur Big Data, Analyste de données, etc."},
    {"question": "L'ENIAD a-t-elle un cluster Hadoop ?", "réponse": "Oui, un cluster Hadoop est disponible pour les projets étudiants."},
    {"question": "Comment participer à un hackathon Big Data ?", "réponse": "Inscrivez-vous aux événements annoncés sur le portail ou par email."},
    {"question": "Qu'est-ce qu'une base de données vectorielle ?", "réponse": "C'est une base optimisée pour les recherches rapides, utilisée dans le Big Data et l'IA."},
    {"question": "Le Big Data est-il enseigné en anglais ?", "réponse": "Oui, certains cours avancés sont en anglais pour attirer un public international."},
    {"question": "Comment stocker des données massives à l'ENIAD ?", "réponse": "Les données sont stockées sur des serveurs cloud ou locaux avec Hadoop HDFS."},
    {"question": "Y a-t-il des partenariats Big Data avec des entreprises ?", "réponse": "Oui, des collaborations existent avec des sociétés technologiques marocaines et internationales."},
    {"question": "Quels sont les projets Big Data en cours ?", "réponse": "Des projets sur l'analyse de données éducatives et industrielles sont en cours."},
    {"question": "Comment apprendre Apache Spark à l'ENIAD ?", "réponse": "Inscrivez-vous au module Big Data ou suivez les tutoriels en ligne de l'ENIAD."},
    {"question": "Le Big Data aide-t-il les étudiants à l'ENIAD ?", "réponse": "Oui, il permet d'analyser des données pour améliorer les services et l'enseignement."}
]


In [ ]:
# Configuration du logging
logging.basicConfig(level=logging.INFO, 
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    filename='eniad_assistant.log')
logger = logging.getLogger(__name__)

In [ ]:
# Chargement des données
try:
    with open('data/questions_fr.json', 'r', encoding='utf-8') as f:
        qa_data_fr = json.load(f)
    
    with open('data/questions_en.json', 'r', encoding='utf-8') as f:
        qa_data_en = json.load(f)
    
    logger.info("Données chargées avec succès")
except Exception as e:
    logger.error(f"Erreur lors du chargement des données: {e}")
    # Données de secours
    qa_data_fr = [
        {"question": "Quels sont les horaires d'ouverture de la bibliothèque de l'ENIAD Berkane ?", 
         "réponse": "La bibliothèque est ouverte de 8h à 20h du lundi au vendredi, et de 9h à 17h le samedi."},
        {"question": "Comment puis-je m'inscrire à un club étudiant ?", 
         "réponse": "Pour vous inscrire à un club étudiant, rendez-vous au bureau de la vie étudiante avec votre carte d'étudiant ou visitez le portail en ligne des activités étudiantes."}
    ]
    qa_data_en = [
        {"question": "What are the opening hours of the ENIAD Berkane library?", 
         "answer": "The library is open from 8am to 8pm Monday to Friday, and from 9am to 5pm on Saturday."},
        {"question": "How can I join a student club?", 
         "answer": "To join a student club, go to the student life office with your student ID or visit the online student activities portal."}
    ]

In [ ]:
# Initialisation de la base de données vectorielle
def initialize_vector_db():
    try:
        # Préparation des documents
        documents_fr = [doc["question"] + " " + doc["réponse"] for doc in qa_data_fr]
        documents_en = [doc["question"] + " " + doc["answer"] for doc in qa_data_en]
        
        all_documents = documents_fr + documents_en
        
        # Utilisation des embeddings de HuggingFace
        embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
        
        # Création de la base de données vectorielle
        vector_db = FAISS.from_texts(all_documents, embeddings)
        
        logger.info("Base de données vectorielle initialisée avec succès")
        return vector_db
    except Exception as e:
        logger.error(f"Erreur lors de l'initialisation de la base de données vectorielle: {e}")
        return None

In [ ]:
# Initialisation du modèle de langage
def initialize_llm():
    try:
        # Pour l'usage en production, utilisez votre API OpenAI ou un autre modèle
        # llm = OpenAI(temperature=0.7, model_name="gpt-3.5-turbo")
        
        # Version alternative avec HuggingFace
        llm = HuggingFaceHub(repo_id="mistralai/Mistral-7B-Instruct-v0.2", 
                            model_kwargs={"temperature": 0.7, "max_length": 512})
        
        logger.info("Modèle de langage initialisé avec succès")
        return llm
    except Exception as e:
        logger.error(f"Erreur lors de l'initialisation du modèle de langage: {e}")
        return None

In [ ]:
# Initialisation de la mémoire de conversation
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Initialisation des ressources
vector_db = initialize_vector_db()
llm = initialize_llm()

# Création de la chaîne de conversation
if vector_db and llm:
    qa_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vector_db.as_retriever(search_kwargs={"k": 3}),
        memory=memory,
        return_source_documents=True
    )

In [ ]:
# Traduction
def translate_text(text, target_lang):
    try:
        translator = Translator(to_lang=target_lang)
        return translator.translate(text)
    except Exception as e:
        logger.error(f"Erreur lors de la traduction: {e}")
        return text

In [ ]:
# Reconnaissance d'entités nommées
ner_pipeline = pipeline("ner")
def extract_entities(text):
    try:
        entities = ner_pipeline(text)
        return entities
    except Exception as e:
        logger.error(f"Erreur lors de l'extraction d'entités: {e}")
        return []

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision 4c53496 (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


RuntimeError: Failed to import transformers.models.bert.modeling_tf_bert because of the following error (look up to see its traceback):
Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.